In [57]:
import numpy as np
import pandas as pd 

from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error

import eli5
from eli5.sklearn import PermutationImportance


In [7]:
cd data

C:\Users\benia\Desktop\Machine_Learning\Prediction price cars\data


In [58]:
df = pd.read_hdf('car.h5')

In [9]:
df.head()

,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,feature_wspomaganie-kierownicy,feature_podgrzewana-przednia-szyba,feature_przyciemniane-szyby,feature_elektrycznie-ustawiane-fotele,feature_klimatyzacja-czterostrefowa,feature_tuner-tv,feature_poduszki-boczne-przednie,feature_tempomat-aktywny,feature_klimatyzacja-automatyczna,feature_poduszki-boczne-tylne,feature_radio-fabryczne,feature_odtwarzacz-dvd,feature_czujniki-parkowania-tylne,feature_łopatki-zmiany-biegów,feature_centralny-zamek,feature_dach-panoramiczny,feature_kamera-cofania,feature_immobilizer,feature_czujnik-martwego-pola,feature_gniazdo-sd,...,param_metallic,param_leasing-concession,param_color,param_financing-option,param_original-owner,param_vat-marża,param_kategoria,param_co2-emissions,param_leasing,param_mileage,param_zarejestrowany-jako-zabytek,param_napęd,param_wartość-wykupu,param_typ,param_metalik,param_pierwszy-właściciel,param_skrzynia-biegów,param_vat-discount,param_no-accident,param_liczba-pozostałych-rat,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
0,"[Osobowe, Dacia, Logan, Dacia Logan II (2012-)]","13:23, 5 marca 2018",PLN,Cena Brutto,45100.0,"Krasne 7A - 36-007 Krasne, rzeszowski, Podkarp...",AUTO SPEKTRUM,Dealer,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,...,None,None,None,None,None,None,Osobowe,None,None,None,None,None,None,Sedan,Tak,None,Manualna,None,None,None,None,None,Firmy,90 KM,None,None,None,None,None,Benzyna+LPG,None,None,None,None,None,None,None,Logan,898 cm3,None,None,None,None,None,None,None,None,None,Inny kolor,None,None,Nowe,II (2012-),None,None,None,Tak,None,None,0
2,"[Osobowe, Peugeot, 3008, Peugeot 3008 I (2009-...","15:48, 5 marca 2018",PLN,"Cena Brutto, Do negocjacji, Faktura VAT",28000.0,"Tomaszów Mazowiecki, tomaszowski, Łódzkie",None,Osoba prywatna,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,True,False,False,...,

In [59]:
SUFFIX_CAT = "_cat"
for feat in df.columns:
    if isinstance(df[feat][0],list): continue
    
    factorized_values = df[feat].factorize()[0]
    
    if SUFFIX_CAT in feat:
        df[feat] = factorized_values
    else:
        df[feat + SUFFIX_CAT] = factorized_values


In [60]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [61]:
#X = df[['param_faktura-vat_cat','param_faktura-vat_cat','param_stan_cat','param_rok-produkcji_cat','param_moc_cat','feature_kamera-cofania_cat']].values
def run_model(model , feats):
    X = df[feats].values
    y = df["price_value"].values
    
    scores = cross_val_score(model,X,y,cv=3,scoring="neg_mean_absolute_error")
    return np.mean(scores), np.std(scores)

In [55]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19747.810938471786, 75.95212671816479)

In [36]:
model = RandomForestRegressor(max_depth=5,n_estimators=50, random_state=0)
run_model(model,cat_feats)

(-18864.27843495869, 38.24233578175403)

In [40]:
xgb_params={
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}
run_model(xgb.XGBRegressor(**xgb_params),cat_feats)

(-13034.249470063352, 103.73566521784606)

In [51]:
len(cat_feats)

151

In [65]:
feats =['param_napęd_cat','param_rok-produkcji_cat','param_stan_cat','param_faktura-vat_cat','param_moc_cat','param_skrzynia-biegów_cat','feature_kamera-cofania_cat','param_marka-pojazdu_cat','param_typ_cat','param_pojemność-skokowa_cat','feature_wspomaganie-kierownicy_cat','seller_name_cat','param_wersja_cat','param_model-pojazdu_cat','feature_światła-led_cat','feature_asystent-pasa-ruchu_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_regulowane-zawieszenie_cat','feature_łopatki-zmiany-biegów_cat']
run_model(xgb.XGBRegressor(**xgb_params),feats)

(-13585.191544077223, 107.2464275889594)

In [67]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== "None" else int(x))

In [73]:
feats =['param_napęd_cat','param_rok-produkcji','param_stan_cat','param_faktura-vat_cat','param_moc','param_skrzynia-biegów_cat','feature_kamera-cofania_cat','param_marka-pojazdu_cat','param_typ_cat','param_pojemność-skokowa_cat','feature_wspomaganie-kierownicy_cat','seller_name_cat','param_wersja_cat','param_model-pojazdu_cat','feature_światła-led_cat','feature_asystent-pasa-ruchu_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_regulowane-zawieszenie_cat','feature_łopatki-zmiany-biegów_cat']
run_model(xgb.XGBRegressor(**xgb_params),feats)

(-9772.408396613564, 85.3312046882783)

In [71]:
df['param_moc']= df['param_moc'].map(lambda x: -1 if str(x)== "None" else int(x.split(' ')[0]))

In [72]:
df['param_moc']

0          90
2         115
3         262
4         110
5         310
         ... 
160609     75
160610     75
160611    120
160614    150
160615     88
Name: param_moc, Length: 106494, dtype: int64

In [77]:
df['param_pojemność-skokowa']= df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)== "None" else int(str(x).split('cm')[0].replace(' ','')))

In [78]:
df['param_pojemność-skokowa']


0          898
2         1560
3         3000
4         1560
5         1984
          ... 
160609    1120
160610    1200
160611    1900
160614    2500
160615    1360
Name: param_pojemność-skokowa, Length: 106494, dtype: int64

In [76]:
feats =['param_napęd_cat','param_rok-produkcji','param_stan_cat','param_faktura-vat_cat','param_moc','param_skrzynia-biegów_cat','feature_kamera-cofania_cat','param_marka-pojazdu_cat','param_typ_cat','param_pojemność-skokowa','feature_wspomaganie-kierownicy_cat','seller_name_cat','param_wersja_cat','param_model-pojazdu_cat','feature_światła-led_cat','feature_asystent-pasa-ruchu_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_regulowane-zawieszenie_cat','feature_łopatki-zmiany-biegów_cat']
run_model(xgb.XGBRegressor(**xgb_params),feats)

(-9618.08167109244, 79.8982870999676)